# AutoML: Train "the best" classifier model for the UCI Bank Marketing dataset. 

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)
- The latest MLFlow packages:

    pip install azureml-mlflow

    pip install mlflow

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create an `AutoML classification Job` with the 'classification()' factory-fuction.
- Submit and run the AutoML classification job
- Obtaing the model and score predictions with it

**Motivations** - This notebook explains how to setup and run an AutoML classification job. This is one of the nine ML-tasks supported by AutoML. Other ML-tasks are 'regression', 'time-series forecasting', 'image classification', 'image object detection', 'nlp text classification', etc.

In this example we use the UCI Bank Marketing dataset to showcase how you can use AutoML for a classification problem. The classification goal is to predict if the client will subscribe to a term deposit with the bank.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [7]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.identity import InteractiveBrowserCredential
from azure.ml import MLClient

from azure.ml._constants import AssetTypes
from azure.ml import automl
from azure.ml.entities import JobInput

## 1.2. Workspace details

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [interactive authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.interactivebrowsercredential?view=azure-python) for this tutorial. More advanced connection methods can be found [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

By default, we try to use the by default workspace configuration (available out-of-the-box in Compute Instances) or from any Config.json file you might have copied into the folders structure.
If no Config.json is found, then you need to manually introduce the subscription_id, resource_group and workspace when creating MLClient .

In [8]:
credential = InteractiveBrowserCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "your-azure-subscription-guid-id"
    resource_group = "your-azure-resource-group-name"
    workspace = "your-azure-machine-lenaring-workspace-name"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

Found the config file in: C:\Users\CESARDL.CESARDLSB2-MSFT\GitAMLRepos\azureml-examples-automl-preview-branch\sdk\jobs\automl-standalone-jobs\config.json


# 2. Configure and run the AutoML classification job
In this section we will configure and run the AutoML classification job.

## 2.1 Configure the job through the classification() factory function

### classification() parameters:

The `classification()` factory function allows user to configure AutoML for the classification task for the most common scenarios with the following properties.

- `target_column_name` - The name of the column to target for predictions. It must always be specified. This parameter is applicable to 'training_data', 'validation_data' and 'test_data'.
- `primary_metric` - The metric that AutoML will optimize for Classification model selection.
- `training_data` - The data to be used for training. It should contain both training feature columns and a target column. Optionally, this data can be split for segregating a validation or test dataset. 
You can use a registered MLTable in the workspace using the format '<mltable_name>:<version>' OR you can use a local file or folder as a MLTable. For e.g JobInput(mltable='my_mltable:1') OR JobInput(mltable=MLTable(local_path="./data"))
The parameter 'training_data' must always be provided.
- `compute` - The compute on which the AutoML job will run. In this example we are using a compute called 'cpu-cluster' present in the workspace. You can replace it any other compute in the workspace. 
- `name` - The name of the Job/Run. This is an optional property. If not specified, a random name will be generated.
- `experiment_name` - The name of the Experiment. An Experiment is like a folder with multiple runs in Azure ML Workspace that should be related to the same logical machine learning experiment.

### set_limits() parameters:
This is an optional configuration method to configure limits parameters such as timeouts.     
    
- timeout_minutes - Maximum amount of time in minutes that the whole AutoML job can take before the job terminates. This timeout includes setup, featurization and training runs but does not include the ensembling and model explainability runs at the end of the process since those actions need to happen once all the trials (children jobs) are done. If not specified, the default job's total timeout is 6 days (8,640 minutes). To specify a timeout less than or equal to 1 hour (60 minutes), make sure your dataset's size is not greater than 10,000,000 (rows times column) or an error results.

- trial_timeout_minutes - Maximum time in minutes that each trial (child job) can run for before it terminates. If not specified, a value of 1 month or 43200 minutes is used.
    
- max_trials - The maximum number of trials/runs each with a different combination of algorithm and hyperparameters to try during an AutoML job. If not specified, the default is 1000 trials. If using 'enable_early_termination' the number of trials used can be smaller.
    
- max_concurrent_trials - Represents the maximum number of trials (children jobs) that would be executed in parallel. It's a good practice to match this number with the number of nodes your cluster.
    
- enable_early_termination - Whether to enable early termination if the score is not improving in the short term. 
    

In [9]:
# Create MLTables for training dataset

my_training_data_input = JobInput(
    type=AssetTypes.MLTABLE, path="./data/training-mltable-folder"
)

# Remote MLTable definition
# my_training_data_input  = JobInput(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/Classification/Train")

In [10]:
# General job parameters
compute_name = "cpu-cluster"
max_trials = 5
exp_name = "dpv2-classifier-experiment"

In [11]:
# Create the AutoML classification job with the related factory-function.

classification_job = automl.classification(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    target_column_name="y",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True,
    tags={"my_custom_tag": "My custom value"},
    # These are temporal properties needed in Private Preview
    properties={
        "_automl_internal_label": "latest",
        "_automl_internal_save_mlflow": True,
    },
)

# Limits are all optional
classification_job.set_limits(
    timeout=600,  # timeout_minutes
    trial_timeout=20,  # trial_timeout_minutes
    max_trials=max_trials,
    # max_concurrent_trials = 4,
    # max_cores_per_trial: -1,
    enable_early_termination=True,
)

# Training properties are optional
classification_job.set_training(
    blocked_models=["LogisticRegression"], 
    enable_onnx_compatible_models=True
)

# Featurization properties are optional
# classification_job.set_featurization(# drop_columns=["not_needed_column"], # Optional
#                                      # enable_dnn_featurization=True         # Enable if there are text columns
#                                     )


## 2.2 Run the CommandJob
Using the `MLClient` created earlier, we will now run this CommandJob in the workspace.

In [12]:
# Submit the AutoML job (CDLTLL: Is it ml_client.create_or_update(classification_job))
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

# Get a URL for the status of the job
# returned_job.services["Studio"].endpoint

Created job: ClassificationJob({'log_verbosity': 'Info', 'task_type': <TaskType.CLASSIFICATION: 'Classification'>, 'environment_id': None, 'environment_variables': None, 'outputs': {}, 'display_name': 'f9cf29cd-30f1-4289-ab00-b8fcb65c9cd9', 'type': 'automl', 'status': 'NotStarted', 'log_files': None, 'name': 'f9cf29cd-30f1-4289-ab00-b8fcb65c9cd9', 'description': None, 'tags': {'my_custom_tag': 'My custom value'}, 'properties': {'_automl_internal_label': 'latest', '_automl_internal_save_mlflow': 'True', 'mlflow.source.git.repoURL': 'git@github.com:Azure/azureml-examples.git', 'mlflow.source.git.branch': 'automl-preview', 'mlflow.source.git.commit': 'ecd27b43e6cc039a8405621ec26dc3330fefe590', 'azureml.git.dirty': 'True'}, 'id': '/subscriptions/102a16c3-37d3-48a8-9237-4c9b1e8e80e0/resourceGroups/automlpmdemo/providers/Microsoft.MachineLearningServices/workspaces/cesardl-automl-centraluseuap-ws/jobs/f9cf29cd-30f1-4289-ab00-b8fcb65c9cd9', 'base_path': './', 'creation_context': <azure.ml._re

### Wait until the AutoML job is finished
ml_client.jobs.stream(returned_job.name) waits until the specified job is finished

In [13]:
ml_client.jobs.stream(returned_job.name)

RunId: f9cf29cd-30f1-4289-ab00-b8fcb65c9cd9
Web View: https://ml.azure.com/runs/f9cf29cd-30f1-4289-ab00-b8fcb65c9cd9?wsid=/subscriptions/102a16c3-37d3-48a8-9237-4c9b1e8e80e0/resourcegroups/automlpmdemo/workspaces/cesardl-automl-centraluseuap-ws

Execution Summary
RunId: f9cf29cd-30f1-4289-ab00-b8fcb65c9cd9
Web View: https://ml.azure.com/runs/f9cf29cd-30f1-4289-ab00-b8fcb65c9cd9?wsid=/subscriptions/102a16c3-37d3-48a8-9237-4c9b1e8e80e0/resourcegroups/automlpmdemo/workspaces/cesardl-automl-centraluseuap-ws



In [14]:
print(returned_job.name)

f9cf29cd-30f1-4289-ab00-b8fcb65c9cd9


# 3. Retrieve the Best Trial (Best Model's trial/run)
Use the MLFLowClient to access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Trial.

## 1.4 Initialize MLFlow Client
The models and artifacts that are produced by AutoML can be accessed via the MLFlow interface. 
Initialize the MLFlow client here, and set the backend as Azure ML, via. the MLFlow Client.

*IMPORTANT*, you need to have installed the latest MLFlow packages with:

    pip install azureml-mlflow

    pip install mlflow

### Obtain the tracking URI for MLFlow

In [15]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(name=ml_client.workspace_name).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

azureml://master.api.azureml-test.ms/mlflow/v1.0/subscriptions/102a16c3-37d3-48a8-9237-4c9b1e8e80e0/resourceGroups/automlpmdemo/providers/Microsoft.MachineLearningServices/workspaces/cesardl-automl-centraluseuap-ws


In [16]:
# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))


Current tracking uri: azureml://master.api.azureml-test.ms/mlflow/v1.0/subscriptions/102a16c3-37d3-48a8-9237-4c9b1e8e80e0/resourceGroups/automlpmdemo/providers/Microsoft.MachineLearningServices/workspaces/cesardl-automl-centraluseuap-ws


In [17]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

# Set the experiment name in MLFlow
# Experiment Name is used in MLFlow and AutoML Job
mlflow.set_experiment(exp_name)

<Experiment: artifact_location='', experiment_id='eb578303-d89f-44ff-b8c8-c22a425836c8', lifecycle_stage='active', name='dpv2-classifier-experiment', tags={}>

### Get the AutoML parent Job

In [18]:
job_name = returned_job.name

# Example if providing an specific Job name/ID
# job_name = "c15b6d6f-969b-40b4-b3d7-2b2ef9d26226"

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

Parent Run: 
<Run: data=<RunData: metrics={'AUC_macro': 0.9487479997886753,
 'AUC_micro': 0.9809116178716071,
 'AUC_weighted': 0.9487479997886753,
 'accuracy': 0.9153636939489914,
 'average_precision_score_macro': 0.8260975078408814,
 'average_precision_score_micro': 0.9817439238532364,
 'average_precision_score_weighted': 0.9557428469205848,
 'balanced_accuracy': 0.7551604589184657,
 'f1_score_macro': 0.7729865175214717,
 'f1_score_micro': 0.9153636939489914,
 'f1_score_weighted': 0.9122845562676405,
 'log_loss': 0.17351885669678385,
 'matthews_correlation': 0.5488583499980033,
 'norm_macro_recall': 0.5103209178369312,
 'precision_score_macro': 0.7952805029004464,
 'precision_score_micro': 0.9153636939489914,
 'precision_score_weighted': 0.9103619729299698,
 'recall_score_macro': 0.7551604589184657,
 'recall_score_micro': 0.9153636939489914,
 'recall_score_weighted': 0.9153636939489914,
 'weighted_accuracy': 0.9553816875537935}, params={}, tags={'automl_best_child_run_id': 'f9cf29cd-3

In [19]:
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(mlflow_parent_run.data.tags)

{'my_custom_tag': 'My custom value', 'model_explain_run': 'best_run', 'pipeline_id': '<5dfac790c5c209f98a1da2dc1c7fb76f0397324f>;<c7af0367625be6ac5c2fecbfc72ed444cb7a2111>;<8b44a8376e136a1410d110171ab167611306916c>;<__AutoML_Ensemble__>;<__AutoML_Stack_Ensemble__>;', 'score': '<0.9148970272823249>;<0.9118635939323221>;<0.9000298994276823>;<0.9153636939489914>;<0.9146970328388064>;', 'predicted_cost': '<0>;<0>;<0.5>;<0>;<0>;', 'fit_time': '<0.220256>;<1.0037496>;<5.8594106>;<14>;<22>;', 'training_percent': '<100>;<100>;<100>;<100>;<100>;', 'iteration': '<0>;<1>;<2>;<3>;<4>;', 'run_preprocessor': '<MaxAbsScaler>;<MaxAbsScaler>;<StandardScalerWrapper>;<>;<>;', 'run_algorithm': '<LightGBM>;<XGBoostClassifier>;<ExtremeRandomTrees>;<VotingEnsemble>;<StackEnsemble>;', 'automl_best_child_run_id': 'f9cf29cd-30f1-4289-ab00-b8fcb65c9cd9_3', 'mlflow.rootRunId': 'f9cf29cd-30f1-4289-ab00-b8fcb65c9cd9', 'mlflow.runName': 'f9cf29cd-30f1-4289-ab00-b8fcb65c9cd9'}


## Get the AutoML best child run

In [20]:
# Get the best model's child run

best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

Found best child run id:  f9cf29cd-30f1-4289-ab00-b8fcb65c9cd9_3
Best child run: 
<Run: data=<RunData: metrics={'AUC_macro': 0.9487479997886753,
 'AUC_micro': 0.9809116178716071,
 'AUC_weighted': 0.9487479997886753,
 'accuracy': 0.9153636939489914,
 'average_precision_score_macro': 0.8260975078408814,
 'average_precision_score_micro': 0.9817439238532364,
 'average_precision_score_weighted': 0.9557428469205848,
 'balanced_accuracy': 0.7551604589184657,
 'f1_score_macro': 0.7729865175214717,
 'f1_score_micro': 0.9153636939489914,
 'f1_score_weighted': 0.9122845562676405,
 'log_loss': 0.17351885669678385,
 'matthews_correlation': 0.5488583499980033,
 'norm_macro_recall': 0.5103209178369312,
 'precision_score_macro': 0.7952805029004464,
 'precision_score_micro': 0.9153636939489914,
 'precision_score_weighted': 0.9103619729299698,
 'recall_score_macro': 0.7551604589184657,
 'recall_score_micro': 0.9153636939489914,
 'recall_score_weighted': 0.9153636939489914,
 'weighted_accuracy': 0.955381

## Get best model run's metrics

Access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Run.

In [21]:
import pandas as pd

pd.DataFrame(best_run.data.metrics, index=[0]).T

,0
recall_score_micro,0.915364
precision_score_micro,0.915364
f1_score_weighted,0.912285
accuracy,0.915364
precision_score_weighted,0.910362
weighted_accuracy,0.955382
AUC_micro,0.980912
AUC_macro,0.948748
norm_macro_recall,0.510321
recall_score_weighted,0.915364


# Next Steps
You can see further examples of other AutoML tasks such as Image-Classification, Image-Object-Detection, NLP-Text-Classification, Time-Series-Forcasting, etc.